In [ ]:
import pandas as pd


In [6]:
import torch
import torch.nn as nn

# ------------------ Base Building Blocks ------------------
class ConvBNSiLU(nn.Module):
    def __init__(self, in_ch, out_ch, k=1, s=1, p=None):
        super().__init__()
        if p is None:
            p = k // 2
        self.conv = nn.Conv2d(in_ch, out_ch, k, s, p, bias=False)
        self.bn = nn.BatchNorm2d(out_ch)
        self.act = nn.SiLU()

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

class Bottleneck(nn.Module):
    def __init__(self, c1, c2):
        super().__init__()
        self.use_shortcut = c1 == c2
        if self.use_shortcut:
            self.conv = ConvBNSiLU(c1, c2, k=1)
        else:
            self.conv1 = ConvBNSiLU(c1, c2, k=1)
            self.conv2 = ConvBNSiLU(c2, c2, k=3)

    def forward(self, x):
        if self.use_shortcut:
            return self.conv(x)
        else:
            x1 = self.conv1(x)
            x2 = self.conv2(x1)
            return torch.cat([x, x2], dim=1)

class SEBlock(nn.Module):
    def __init__(self, in_ch, reduction=16):
        super().__init__()
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(in_ch, in_ch // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(in_ch // reduction, in_ch, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

class C3Block(nn.Module):
    def __init__(self, in_ch, out_ch, n=3):
        super().__init__()
        self.conv1 = ConvBNSiLU(in_ch, in_ch, k=1)
        self.bottlenecks = nn.Sequential(*[Bottleneck(in_ch, in_ch) for _ in range(n)])
        self.concat_conv = ConvBNSiLU(in_ch * 2, out_ch, k=1)
        self.se = SEBlock(out_ch)

    def forward(self, x):
        y1 = self.conv1(x)
        y2 = self.bottlenecks(y1)
        cat = torch.cat((y1, y2), dim=1)
        out = self.concat_conv(cat)
        return self.se(out)

# ------------------ YOLOv5m Architecture (Kidney Stone Detection) ------------------
class YOLOv5mSE(nn.Module):
    def __init__(self, num_classes=1):
        super().__init__()

        # Reduced channel dimensions
        self.stem = ConvBNSiLU(3, 24, k=3, s=1)  # Reduced from 32
        self.stage1 = C3Block(24, 48, n=1)       # Reduced from 64
        self.stage2 = C3Block(48, 96, n=2)       # Reduced from 128, n=2 instead of 3
        self.stage3 = C3Block(96, 192, n=2)      # Reduced from 256, n=2 instead of 3
        self.stage4 = C3Block(192, 384, n=1)     # Reduced from 512

        # Neck with reduced channels
        self.neck1 = ConvBNSiLU(384, 192, k=1)   # Reduced from 256
        self.neck2 = C3Block(192 + 192, 192, n=1)  # Reduced from 256

        self.neck3 = ConvBNSiLU(192, 96, k=1)    # Reduced from 128
        self.neck4 = C3Block(96 + 96, 96, n=1)   # Reduced from 128

        # Head with reduced channels
        self.detect1 = nn.Conv2d(96, (num_classes + 5) * 3, 1)   # Reduced from 128
        self.detect2 = nn.Conv2d(192, (num_classes + 5) * 3, 1)  # Reduced from 256
        self.detect3 = nn.Conv2d(384, (num_classes + 5) * 3, 1)  # Reduced from 512

    def forward(self, x):
        # Backbone
        x = self.stem(x)
        x1 = self.stage1(x)
        x2 = self.stage2(x1)
        x3 = self.stage3(x2)
        x4 = self.stage4(x3)

        # Neck with explicit size matching
        n1 = self.neck1(x4)
        n1_up = nn.functional.interpolate(n1, size=x3.shape[2:], mode='nearest')
        n2 = self.neck2(torch.cat([n1_up, x3], dim=1))

        n2_up = nn.functional.interpolate(self.neck3(n2), size=x2.shape[2:], mode='nearest')
        n3 = self.neck4(torch.cat([n2_up, x2], dim=1))

        # Head
        out1 = self.detect1(n3)
        out2 = self.detect2(n2)
        out3 = self.detect3(x4)

        return out1, out2, out3

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# ----- Base Blocks (same as yours, no change needed) -----
class ConvBNSiLU(nn.Module):
    def __init__(self, in_ch, out_ch, k=1, s=1, p=None):
        super().__init__()
        if p is None:
            p = k // 2
        self.conv = nn.Conv2d(in_ch, out_ch, k, s, p, bias=False)
        self.bn = nn.BatchNorm2d(out_ch)
        self.act = nn.SiLU()

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

class Bottleneck(nn.Module):
    def __init__(self, c1, c2):
        super().__init__()
        self.use_shortcut = c1 == c2
        if self.use_shortcut:
            self.conv = ConvBNSiLU(c1, c2, k=1)
        else:
            self.conv1 = ConvBNSiLU(c1, c2, k=1)
            self.conv2 = ConvBNSiLU(c2, c2, k=3)

    def forward(self, x):
        if self.use_shortcut:
            return self.conv(x)
        else:
            x1 = self.conv1(x)
            x2 = self.conv2(x1)
            return torch.cat([x, x2], dim=1)

class SEBlock(nn.Module):
    def __init__(self, in_ch, reduction=16):
        super().__init__()
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(in_ch, in_ch // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(in_ch // reduction, in_ch, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

class C3Block(nn.Module):
    def __init__(self, in_ch, out_ch, n=1):
        super().__init__()
        self.conv1 = ConvBNSiLU(in_ch, in_ch, k=1)
        self.bottlenecks = nn.Sequential(*[Bottleneck(in_ch, in_ch) for _ in range(n)])
        self.concat_conv = ConvBNSiLU(in_ch * 2, out_ch, k=1)
        self.se = SEBlock(out_ch)

    def forward(self, x):
        y1 = self.conv1(x)
        y2 = self.bottlenecks(y1)
        cat = torch.cat((y1, y2), dim=1)
        out = self.concat_conv(cat)
        return self.se(out)

# ----- YOLOv5nSE Nano Model -----
class YOLOv5nSE(nn.Module):
    def __init__(self, num_classes=1):
        super().__init__()

        # Super lightweight stem and stages
        self.stem = ConvBNSiLU(3, 16, k=3, s=1)
        self.stage1 = C3Block(16, 32, n=1)
        self.stage2 = C3Block(32, 64, n=1)
        self.stage3 = C3Block(64, 128, n=1)
        self.stage4 = C3Block(128, 256, n=1)

        # Neck
        self.neck1 = ConvBNSiLU(256, 128, k=1)
        self.neck2 = C3Block(128 + 128, 128, n=1)

        self.neck3 = ConvBNSiLU(128, 64, k=1)
        self.neck4 = C3Block(64 + 64, 64, n=1)

        # Detection Head
        self.detect1 = nn.Conv2d(64, (num_classes + 5) * 3, 1)
        self.detect2 = nn.Conv2d(128, (num_classes + 5) * 3, 1)
        self.detect3 = nn.Conv2d(256, (num_classes + 5) * 3, 1)

    def forward(self, x):
        # Backbone
        x = self.stem(x)
        x1 = self.stage1(x)
        x2 = self.stage2(x1)
        x3 = self.stage3(x2)
        x4 = self.stage4(x3)

        # Neck
        n1 = self.neck1(x4)
        n1_up = F.interpolate(n1, size=x3.shape[2:], mode='nearest')
        n2 = self.neck2(torch.cat([n1_up, x3], dim=1))

        n2_up = F.interpolate(self.neck3(n2), size=x2.shape[2:], mode='nearest')
        n3 = self.neck4(torch.cat([n2_up, x2], dim=1))

        # Head
        out1 = self.detect1(n3)
        out2 = self.detect2(n2)
        out3 = self.detect3(x4)

        return out1, out2, out3


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
import os
import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm

# -------------------- Dataset with Bilateral Filtering + Augmentation --------------------
class KidneyStoneDataset(Dataset):
    def __init__(self, split_dir, img_size=320):
        self.image_dir = os.path.join(split_dir, "images")
        self.label_dir = os.path.join(split_dir, "labels")
        self.image_files = [f for f in os.listdir(self.image_dir) if f.endswith('.png') or f.endswith('.jpg')]
        self.img_size = img_size
        self.transforms = T.Compose([
            T.RandomHorizontalFlip(p=0.5),
            T.RandomRotation(degrees=10),
            T.ToTensor()
        ])


    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_name)
        label_path = os.path.join(self.label_dir, img_name.replace('.png', '.txt').replace('.jpg', '.txt'))

        # Read image and apply bilateral filter
        img = cv2.imread(img_path)
        img = cv2.resize(img, (self.img_size, self.img_size))
        img = cv2.bilateralFilter(img, d=9, sigmaColor=75, sigmaSpace=75)
        img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        img_tensor = self.transforms(img)

        # Load labels (YOLO format)
        targets = []
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f.readlines():
                    cls, x, y, w, h = map(float, line.strip().split())
                    targets.append([cls, x, y, w, h])

        return img_tensor, torch.tensor(targets), img_name


# -------------------- Custom YOLO Loss --------------------
def bbox_ciou(box1, box2):
    # Compute Complete IoU (CIoU)
    b1_x1, b1_y1 = box1[..., 0] - box1[..., 2] / 2, box1[..., 1] - box1[..., 3] / 2
    b1_x2, b1_y2 = box1[..., 0] + box1[..., 2] / 2, box1[..., 1] + box1[..., 3] / 2
    b2_x1, b2_y1 = box2[..., 0] - box2[..., 2] / 2, box2[..., 1] - box2[..., 3] / 2
    b2_x2, b2_y2 = box2[..., 0] + box2[..., 2] / 2, box2[..., 1] + box2[..., 3] / 2

    inter_rect_x1 = torch.max(b1_x1, b2_x1)
    inter_rect_y1 = torch.max(b1_y1, b2_y1)
    inter_rect_x2 = torch.min(b1_x2, b2_x2)
    inter_rect_y2 = torch.min(b1_y2, b2_y2)
    inter_area = torch.clamp(inter_rect_x2 - inter_rect_x1, min=0) * torch.clamp(inter_rect_y2 - inter_rect_y1, min=0)
    b1_area = (b1_x2 - b1_x1) * (b1_y2 - b1_y1)
    b2_area = (b2_x2 - b2_x1) * (b2_y2 - b2_y1)
    union = b1_area + b2_area - inter_area
    iou = inter_area / (union + 1e-6)

    # center distance
    center_dist = (box1[..., 0] - box2[..., 0]) ** 2 + (box1[..., 1] - box2[..., 1]) ** 2
    # diagonal length of the smallest enclosing box
    enc_x1 = torch.min(b1_x1, b2_x1)
    enc_y1 = torch.min(b1_y1, b2_y1)
    enc_x2 = torch.max(b1_x2, b2_x2)
    enc_y2 = torch.max(b1_y2, b2_y2)
    enc_diag = (enc_x2 - enc_x1) ** 2 + (enc_y2 - enc_y1) ** 2

    v = (4 / (np.pi ** 2)) * torch.pow(torch.atan(box1[..., 2] / box1[..., 3]) - torch.atan(box2[..., 2] / box2[..., 3]), 2)
    alpha = v / (1 - iou + v + 1e-6)
    ciou = iou - center_dist / (enc_diag + 1e-6) - alpha * v
    return ciou


class YoloLoss(torch.nn.Module):
    def __init__(self, lambda_cls=1.0, lambda_obj=1.0, lambda_box=5.0):
        super().__init__()
        self.lambda_cls = lambda_cls
        self.lambda_obj = lambda_obj
        self.lambda_box = lambda_box
        self.bce = torch.nn.BCEWithLogitsLoss()

    def forward(self, preds, targets):
        # Decode preds
        if isinstance(preds, tuple):
            preds = torch.cat([p.view(p.size(0), -1, p.size(-1)) for p in preds], dim=1)

        batch_size = preds.size(0)
        device = preds.device  # Ensure all tensors are on the same device
        loss_cls = torch.tensor(0.0, device=device)
        loss_obj = torch.tensor(0.0, device=device)
        loss_box = torch.tensor(0.0, device=device)

        for b in range(batch_size):
            pred = preds[b]  # [N, 6] or [N, 85] -> assuming [x, y, w, h, obj, cls...]
            t = targets[b]   # [M, 5] -> [cls, x, y, w, h]

            if len(t) == 0:
                continue

            for gt in t:
                gt_cls, gx, gy, gw, gh = gt.to(pred.device)
                gt_box = torch.tensor([gx, gy, gw, gh], device=pred.device)

                ious = bbox_ciou(pred[:, :4], gt_box.unsqueeze(0))  # Shape: [N]
                best_idx = torch.argmax(ious)

                pred_box = pred[best_idx, :4]
                pred_obj = pred[best_idx, 4]
                pred_cls = pred[best_idx, 5:]

                ciou = bbox_ciou(pred_box.unsqueeze(0), gt_box.unsqueeze(0))  # Shape: [1]
                loss_box += (1 - ciou)  # No need to unsqueeze, ciou is already [1]
                loss_obj += self.bce(pred_obj, torch.tensor(1.0, device=pred.device))
                loss_cls += self.bce(pred_cls[int(gt_cls)], torch.tensor(1.0, device=pred.device))

        total_loss = self.lambda_box * loss_box + self.lambda_obj * loss_obj + self.lambda_cls * loss_cls
        return total_loss



# -------------------- Training Function --------------------
def train(model, train_loader, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        # Create progress bar for epochs
        epoch_pbar = tqdm(range(len(train_loader)), 
                         desc=f'Epoch {epoch+1}/{epochs}',
                         position=0)
        
        total_loss = 0
        optimizer.zero_grad()
        
        for i, (imgs, targets, img_names) in enumerate(train_loader):  # Unpack img_names as well
            imgs = imgs.cuda()
            targets = [t.cuda() for t in targets]
            
            outputs = model(imgs)
            loss = criterion(outputs, targets)
            loss = loss / accumulation_steps  # Normalize loss
            loss.backward()
            
            total_loss += loss.item()
            
            if (i + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
            
            # Update progress bar with current loss
            epoch_pbar.set_postfix({
                'loss': f'{total_loss/(i+1):.4f}',
                'batch': f'{i+1}/{len(train_loader)}'
            })
            epoch_pbar.update(1)
        
        epoch_pbar.close()
        avg_loss = total_loss / len(train_loader)
        print(f'\nEpoch {epoch+1}/{epochs} - Average Loss: {avg_loss:.4f}')

def validate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for imgs, targets, _ in dataloader:
            imgs = imgs.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
    return total_loss / len(dataloader)

def custom_collate_fn(batch):
    """
    Custom collate function to handle variable-sized targets
    """
    images = []
    targets = []
    img_names = []
    
    for img, target, img_name in batch:  # Include img_name
        images.append(img)
        targets.append(target)
        img_names.append(img_name)
    
    # Stack images (they should all be the same size)
    images = torch.stack(images)
    
    return images, targets, img_names

# -------------------- Main Script --------------------
if __name__ == "__main__":
    path = os.path.join(os.path.dirname(os.path.abspath('__file__')), '..', 'dataset', 'train')
    dataset = KidneyStoneDataset(path)
    
    # Reduce batch size to save memory
    batch_size = 1  # Reduced from 2
    accumulation_steps = 8  # Increased from 4 to maintain effective batch size
    train_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0,  # Keep this at 0 to prevent memory issues
        collate_fn=custom_collate_fn
    )

    # Enable memory efficient mode
    torch.cuda.empty_cache()  # Clear any unused memory
    
    model = YOLOv5nSE(num_classes=1).cuda()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)
    criterion = YoloLoss()

    # Training with progress bars
    train(model, train_loader, optimizer, epochs=2)



Epoch 1/2:   0%|          | 0/1054 [00:00<?, ?it/s]

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
import os
import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm

# -------------------- Dataset with Bilateral Filtering + Augmentation --------------------
class KidneyStoneDataset(Dataset):
    def __init__(self, split_dir, img_size=320):
        self.image_dir = os.path.join(split_dir, "images")
        self.label_dir = os.path.join(split_dir, "labels")
        self.image_files = [f for f in os.listdir(self.image_dir) if f.endswith('.png') or f.endswith('.jpg')]
        self.img_size = img_size
        self.transforms = T.Compose([
            T.RandomHorizontalFlip(p=0.5),
            T.RandomRotation(degrees=10),
            T.ToTensor()
        ])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_name)
        label_path = os.path.join(self.label_dir, img_name.replace('.png', '.txt').replace('.jpg', '.txt'))

        img = cv2.imread(img_path)
        img = cv2.resize(img, (self.img_size, self.img_size))
        img = cv2.bilateralFilter(img, d=9, sigmaColor=75, sigmaSpace=75)
        img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        img_tensor = self.transforms(img)

        targets = []
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f.readlines():
                    cls, x, y, w, h = map(float, line.strip().split())
                    targets.append([cls, x, y, w, h])

        return img_tensor, torch.tensor(targets), img_name

# -------------------- CIoU + Custom YOLO Loss --------------------
def bbox_ciou(box1, box2):
    b1_x1, b1_y1 = box1[..., 0] - box1[..., 2] / 2, box1[..., 1] - box1[..., 3] / 2
    b1_x2, b1_y2 = box1[..., 0] + box1[..., 2] / 2, box1[..., 1] + box1[..., 3] / 2
    b2_x1, b2_y1 = box2[..., 0] - box2[..., 2] / 2, box2[..., 1] - box2[..., 3] / 2
    b2_x2, b2_y2 = box2[..., 0] + box2[..., 2] / 2, box2[..., 1] + box2[..., 3] / 2

    inter_x1 = torch.max(b1_x1, b2_x1)
    inter_y1 = torch.max(b1_y1, b2_y1)
    inter_x2 = torch.min(b1_x2, b2_x2)
    inter_y2 = torch.min(b1_y2, b2_y2)

    inter_area = torch.clamp(inter_x2 - inter_x1, min=0) * torch.clamp(inter_y2 - inter_y1, min=0)
    b1_area = (b1_x2 - b1_x1) * (b1_y2 - b1_y1)
    b2_area = (b2_x2 - b2_x1) * (b2_y2 - b2_y1)
    union = b1_area + b2_area - inter_area
    iou = inter_area / (union + 1e-6)

    center_dist = (box1[..., 0] - box2[..., 0]) ** 2 + (box1[..., 1] - box2[..., 1]) ** 2
    enc_x1 = torch.min(b1_x1, b2_x1)
    enc_y1 = torch.min(b1_y1, b2_y1)
    enc_x2 = torch.max(b1_x2, b2_x2)
    enc_y2 = torch.max(b1_y2, b2_y2)
    enc_diag = (enc_x2 - enc_x1) ** 2 + (enc_y2 - enc_y1) ** 2

    v = (4 / (np.pi ** 2)) * torch.pow(torch.atan(box1[..., 2] / box1[..., 3]) - torch.atan(box2[..., 2] / box2[..., 3]), 2)
    alpha = v / (1 - iou + v + 1e-6)
    ciou = iou - center_dist / (enc_diag + 1e-6) - alpha * v
    return ciou

class YoloLoss(nn.Module):
    def __init__(self, lambda_cls=1.0, lambda_obj=1.0, lambda_box=5.0):
        super().__init__()
        self.lambda_cls = lambda_cls
        self.lambda_obj = lambda_obj
        self.lambda_box = lambda_box
        self.bce = nn.BCEWithLogitsLoss()

    def forward(self, preds, targets):
        if isinstance(preds, tuple):
            preds = torch.cat([p.view(p.size(0), -1, p.size(-1)) for p in preds], dim=1)

        batch_size = preds.size(0)
        device = preds.device
        loss_cls, loss_obj, loss_box = 0.0, 0.0, 0.0

        for b in range(batch_size):
            pred = preds[b]
            t = targets[b]
            if len(t) == 0:
                continue
            for gt in t:
                gt_cls, gx, gy, gw, gh = gt.to(device)
                gt_box = torch.tensor([gx, gy, gw, gh], device=device)
                ious = bbox_ciou(pred[:, :4], gt_box.unsqueeze(0))
                best_idx = torch.argmax(ious)
                pred_box = pred[best_idx, :4]
                pred_obj = pred[best_idx, 4]
                pred_cls = pred[best_idx, 5:]
                ciou = bbox_ciou(pred_box.unsqueeze(0), gt_box.unsqueeze(0))
                loss_box += (1 - ciou)
                loss_obj += self.bce(pred_obj, torch.tensor(1.0, device=device))
                loss_cls += self.bce(pred_cls[int(gt_cls)], torch.tensor(1.0, device=device))

        total = self.lambda_box * loss_box + self.lambda_obj * loss_obj + self.lambda_cls * loss_cls
        return total

# -------------------- Collate Function --------------------
def custom_collate_fn(batch):
    images, targets, img_names = zip(*batch)
    images = torch.stack(images)
    return images, list(targets), img_names

# -------------------- Training Function --------------------
def train(model, train_loader, optimizer, criterion, epochs, device, accumulation_steps):
    scaler = torch.cuda.amp.GradScaler()
    model.train()
    
    for epoch in range(epochs):
        total_loss = 0
        epoch_pbar = tqdm(range(len(train_loader)), desc=f"Epoch {epoch+1}/{epochs}")
        
        optimizer.zero_grad()
        for i, (imgs, targets, _) in enumerate(train_loader):
            imgs = imgs.to(device)
            targets = [t.to(device) for t in targets]

            with torch.cuda.amp.autocast():
                outputs = model(imgs)
                loss = criterion(outputs, targets)
                loss = loss / accumulation_steps

            scaler.scale(loss).backward()
            total_loss += loss.item()

            if (i + 1) % accumulation_steps == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

            epoch_pbar.set_postfix({
                'loss': f'{total_loss / (i + 1):.4f}',
                'batch': f'{i+1}/{len(train_loader)}'
            })
            epoch_pbar.update(1)

        epoch_pbar.close()
        print(f"\nEpoch {epoch+1} finished - Avg Loss: {total_loss / len(train_loader):.4f}")

# -------------------- Main Execution --------------------
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    path = os.path.join(os.path.dirname(os.path.abspath('__file__')), '..', 'dataset', 'train')
    
    dataset = KidneyStoneDataset(path)
    batch_size = 1
    accumulation_steps = 8
    train_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0,
        pin_memory=True,
        collate_fn=custom_collate_fn
    )

    # 🚨 Replace this with your YOLO model
    model = YOLOv5nSE_Tiny(num_classes=1).to(device)
    
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)
    criterion = YoloLoss().to(device)

    torch.cuda.empty_cache()
    train(model, train_loader, optimizer, criterion, epochs=2, device=device, accumulation_steps=accumulation_steps)


Epoch 1/2:   0%|          | 0/1054 [09:00<?, ?it/s]


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SEBlock(nn.Module):
    def __init__(self, channel, reduction=8):
        super().__init__()
        self.fc = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channel, channel // reduction, 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(channel // reduction, channel, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        weight = self.fc(x)
        return x * weight

class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, se=True):
        super().__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 3, 1, 1)
        self.bn = nn.BatchNorm2d(out_ch)
        self.relu = nn.ReLU(inplace=True)
        self.se = SEBlock(out_ch) if se else nn.Identity()

    def forward(self, x):
        x = self.relu(self.bn(self.conv(x)))
        return self.se(x)

class YOLOv5nSE_Tiny(nn.Module):
    def __init__(self, num_classes=1, num_preds=10):  # num_preds = number of boxes per image
        super().__init__()
        self.layer1 = ConvBlock(3, 16)
        self.pool1 = nn.MaxPool2d(2)

        self.layer2 = ConvBlock(16, 32)
        self.pool2 = nn.MaxPool2d(2)

        self.layer3 = ConvBlock(32, 64)
        self.pool3 = nn.MaxPool2d(2)

        self.layer4 = ConvBlock(64, 128)
        self.pool4 = nn.AdaptiveAvgPool2d((1, 1))

        self.head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, num_preds * (5 + num_classes))  # [x, y, w, h, obj, class_scores]
        )
        self.num_preds = num_preds
        self.num_classes = num_classes

    def forward(self, x):
        x = self.pool1(self.layer1(x))
        x = self.pool2(self.layer2(x))
        x = self.pool3(self.layer3(x))
        x = self.pool4(self.layer4(x))
        x = self.head(x)
        return x.view(x.size(0), self.num_preds, 5 + self.num_classes)  # Shape: [B, N, 6]


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Simple SE Block
class SEBlock(nn.Module):
    def __init__(self, channel, reduction=8):
        super().__init__()
        self.fc = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channel, channel // reduction, 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(channel // reduction, channel, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        weight = self.fc(x)
        return x * weight

# Conv + BN + ReLU + SE (Optional)
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, se=True):
        super().__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 3, 1, 1)
        self.bn = nn.BatchNorm2d(out_ch)
        self.relu = nn.ReLU(inplace=True)
        self.se = SEBlock(out_ch) if se else nn.Identity()

    def forward(self, x):
        x = self.relu(self.bn(self.conv(x)))
        return self.se(x)

# Tiny YOLO-like model
class YOLOv5nSE_Tiny(nn.Module):
    def __init__(self, num_classes=1):
        super().__init__()
        self.layer1 = ConvBlock(3, 16)      # 224 -> 224
        self.pool1 = nn.MaxPool2d(2)        # 224 -> 112

        self.layer2 = ConvBlock(16, 32)
        self.pool2 = nn.MaxPool2d(2)        # 112 -> 56

        self.layer3 = ConvBlock(32, 64)
        self.pool3 = nn.MaxPool2d(2)        # 56 -> 28

        self.layer4 = ConvBlock(64, 128)
        self.pool4 = nn.AdaptiveAvgPool2d((1, 1))  # -> [B, 128, 1, 1]

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes),
            nn.Sigmoid()  # For binary classification
        )

    def forward(self, x):
        x = self.pool1(self.layer1(x))
        x = self.pool2(self.layer2(x))
        x = self.pool3(self.layer3(x))
        x = self.pool4(self.layer4(x))
        x = self.classifier(x)
        return x


CUDA available: True


CUDA available: True


In [7]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as T

class KidneyStoneClassificationDataset(Dataset):
    def __init__(self, data_dir, img_size=224):
        self.image_dir = os.path.join(data_dir, "images")
        self.label_dir = os.path.join(data_dir, "labels")
        self.image_files = [f for f in os.listdir(self.image_dir) if f.endswith(('.jpg', '.png'))]
        self.transforms = T.Compose([
            T.Resize((img_size, img_size)),
            T.RandomHorizontalFlip(),
            T.RandomRotation(10),
            T.ToTensor(),
            T.Normalize(mean=[0.5], std=[0.5])
        ])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_name)
        label_path = os.path.join(self.label_dir, img_name.replace('.png', '.txt').replace('.jpg', '.txt'))

        img = Image.open(img_path).convert('L')  # grayscale
        img = self.transforms(img)

        with open(label_path, 'r') as f:
            label = int(f.readline().strip())

        return img, torch.tensor(label, dtype=torch.float32)


In [19]:
import torch.nn as nn

class KidneyStoneCNN(nn.Module):
    def __init__(self):
        super(KidneyStoneCNN, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * 28 * 28, 128), nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        return self.net(x).squeeze(1)  # for BCEWithLogitsLoss


In [11]:
import torch
from torch.utils.data import DataLoader
from torch import nn, optim
from tqdm import tqdm
import os
def train_model(model):
    # 1. Config
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    batch_size = 16
    epochs = 15
    learning_rate = 1e-3
    path = os.path.join(os.path.dirname(os.path.abspath('__file__')), '..', 'dataset')
    train_path = os.path.join(path, 'train')
    valid_path = os.path.join(path, 'valid')
    # 2. Data Loading
    train_dataset = KidneyStoneClassificationDataset(train_path)
    val_dataset = KidneyStoneClassificationDataset(valid_path)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # 3. Model Setup
    model = model
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # 4. Training Loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        print(f"\n🌀 Epoch {epoch+1}/{epochs}")
        for imgs, labels in tqdm(train_loader, desc="Training"):
            imgs, labels = imgs.to(device), labels.to(device)

            # Forward pass
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_loss = running_loss / len(train_loader)
        print(f"✅ Training Loss: {avg_loss:.4f}")

        # Validation after each epoch
        validate(model, val_loader, criterion, device)

    # Save the model
    torch.save(model.state_dict(), "kidney_stone_cnn.pth")
    print("\n📦 Model saved as 'kidney_stone_cnn.pth'")


def validate(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)

            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            preds = (torch.sigmoid(outputs) > 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    avg_loss = val_loss / len(val_loader)
    print(f"📊 Validation Loss: {avg_loss:.4f} | Accuracy: {accuracy:.2f}%")


if __name__ == "__main__":
    train_model(model)


Ultralytics 8.3.146  Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2050, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov5n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True,

RuntimeError: Dataset 'coco.yaml' error  [WinError 5] Access is denied: 'C:\\Users\\datasets'

In [3]:
!pip install --user ultralytics

  Using cached ultralytics-8.3.146-py3-none-any.whl.metadata (37 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
Using cached ultralytics-8.3.146-py3-none-any.whl (1.0 MB)
Using cached ultralytics_thop-2.0.14-py3-none-any.whl (26 kB)

   ---------------------------------------- 0/2 [ultralytics-thop]
   ---------------------------------------- 0/2 [ultralytics-thop]
   ---------------------------------------- 0/2 [ultralytics-thop]
   ---------------------------------------- 0/2 [ultralytics-thop]
   -------------------- ------------------- 1/2 [ultralytics]
   -------------------- ------------------- 1/2 [ultralytics]
   -------------------- ------------------- 1/2 [ultralytics]
   -------------------- ------------------- 1/2 [ultralytics]
   -------------------- ------------------- 1/2 [ultralytics]
   -------------------- ------------------- 1/2 [ultralytics]
   -------------------- ------------------- 1/2 [ultralytics]
   -------------------- ---------

In [4]:
from ultralytics import YOLO

# Load a COCO-pretrained YOLOv5n model
model = YOLO("yolov5n.pt")

# Display model information (optional)
model.info()

# Train the model on the COCO8 example dataset for 100 epochs
#results = model.train(data="coco8.yaml", epochs=100, imgsz=640)

# Run inference with the YOLOv5n model on the 'bus.jpg' image
#results = model("path/to/bus.jpg")

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\Yaswanth Reddy\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
PRO TIP  Replace 'model=yolov5n.pt' with new 'model=yolov5nu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



100%|██████████| 5.31M/5.31M [00:03<00:00, 1.56MB/s]


YOLOv5n summary: 153 layers, 2,654,816 parameters, 0 gradients, 7.8 GFLOPs


(153, 2654816, 0, 7.840102399999999)

In [14]:
from ultralytics import YOLO

def train_yolo(model):
    model = model or YOLO("yolov8n.pt")  # Load YOLOv8n model if not provided

    model.train(
        data='kidney.yaml',
        epochs=30,
        imgsz=320,
        batch=8,
        device=0 if torch.cuda.is_available() else 'cpu',
        project='kidney-stone-yolo',
        name='yolov8n-classifier',
        pretrained=True
    )

    print("🎉 Training complete! Weights saved in 'runs/detect/yolov8n-classifier'")
    
if __name__ == "__main__":
    train_yolo(model)


Ultralytics 8.3.146  Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2050, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=kidney.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov5n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n-classifier3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretr

100%|██████████| 755k/755k [00:00<00:00, 1.42MB/s]

Overriding model.yaml nc=80 with nc=1



                   from  n    params  module                                       arguments                     
  0                  -1  1      1760  ultralytics.nn.modules.conv.Conv             [3, 16, 6, 2, 2]              
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      4800  ultralytics.nn.modules.block.C3              [32, 32, 1]                   
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     29184  ultralytics.nn.modules.block.C3              [64, 64, 2]                   
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  3    156928  ultralytics.nn.modules.block.C3              [128, 128, 3]                 
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

100%|██████████| 5.35M/5.35M [00:08<00:00, 653kB/s]


...

      30/30     0.344G      1.867     0.8408     0.9861          9        320: 100%|██████████| 132/132 [00:21<00:00,  6.09it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  2.91it/s]

                   all        123        325      0.733      0.566      0.586      0.226



30 epochs completed in 0.145 hours.
Optimizer stripped from kidney-stone-yolo\yolov8n-classifier3\weights\last.pt, 5.2MB
Optimizer stripped from kidney-stone-yolo\yolov8n-classifier3\weights\best.pt, 5.2MB

Validating kidney-stone-yolo\yolov8n-classifier3\weights\best.pt...
Ultralytics 8.3.146  Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2050, 4096MiB)
YOLOv5n summary (fused): 84 layers, 2,503,139 parameters, 0 gradients, 7.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:03<00:00,  2.34it/s]


                   all        123        325      0.748      0.585      0.605      0.237
Speed: 0.5ms preprocess, 6.7ms inference, 0.0ms loss, 8.6ms postprocess per image
Results saved to kidney-stone-yolo\yolov8n-classifier3
🎉 Training complete! Weights saved in 'runs/detect/yolov8n-classifier'


In [40]:
import cv2
from ultralytics import YOLO

# Load your trained model
model = YOLO(r'C:\Users\Yaswanth Reddy\KidneyStoneDetection\model\kidney-stone-yolo\yolov8s-classifier2\weights\best.pt')

# Path to input image
image_path = r'C:\Users\Yaswanth Reddy\KidneyStoneDetection\dataset\valid\images\1-3-46-670589-33-1-63705534438072088700001-5205898188246698645_png_jpg.rf.75a856d4d26f9f0323b495186912084d.jpg'  # 🔁 Change this to your image

# Run inference
results = model(image_path)

# Parse results
for result in results:
    boxes = result.boxes  # Boxes object
    img = result.orig_img.copy()

    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box
        conf = box.conf[0].item()               # Confidence
        label = result.names[int(box.cls[0])]   # Class name

        # Draw rectangle and label
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(img, f'{label} {conf:.2f}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Show image
    cv2.imshow("Detected Stones", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Save output
    cv2.imwrite("output_with_stones.png", img)



image 1/1 C:\Users\Yaswanth Reddy\KidneyStoneDetection\dataset\valid\images\1-3-46-670589-33-1-63705534438072088700001-5205898188246698645_png_jpg.rf.75a856d4d26f9f0323b495186912084d.jpg: 288x320 7 kidney_stones, 67.9ms
Speed: 2.6ms preprocess, 67.9ms inference, 4.5ms postprocess per image at shape (1, 3, 288, 320)


In [36]:
from ultralytics import YOLO
import cv2
import os

model = YOLO(r'C:\Users\Yaswanth Reddy\KidneyStoneDetection\model\kidney-stone-yolo\yolov8n-classifier3\weights\best.pt')

# Run on all images in test folder
test_folder = r'C:\Users\Yaswanth Reddy\KidneyStoneDetection\dataset\test\images'
for img_name in os.listdir(test_folder):
    if img_name.endswith('.jpg') or img_name.endswith('.png'):
        img_path = os.path.join(test_folder, img_name)
        results = model(img_path)
        # Save results to a text file
        with open(os.path.join(test_folder, img_name.replace('.jpg', '.txt').replace('.png', '.txt')), 'w') as f:
            for result in results:
                for box in result.boxes:
                    cls = int(box.cls[0])
                    x, y, w, h = box.xywh[0]
                    f.write(f"{cls} {x} {y} {w} {h}\n")



image 1/1 C:\Users\Yaswanth Reddy\KidneyStoneDetection\dataset\test\images\1-3-46-670589-33-1-63703718086120120200001-5487554579919763006_png_jpg.rf.9fd67251e99a47dbe83a5db6efe6c016.jpg: 288x320 2 kidney_stones, 20.9ms
Speed: 2.7ms preprocess, 20.9ms inference, 3.9ms postprocess per image at shape (1, 3, 288, 320)

image 1/1 C:\Users\Yaswanth Reddy\KidneyStoneDetection\dataset\test\images\1-3-46-670589-33-1-63705534438365105500001-5275982036206127404_png_jpg.rf.365c4daf2b772012fe47e07b9daec86e.jpg: 288x320 1 kidney_stone, 19.4ms
Speed: 2.5ms preprocess, 19.4ms inference, 2.4ms postprocess per image at shape (1, 3, 288, 320)

image 1/1 C:\Users\Yaswanth Reddy\KidneyStoneDetection\dataset\test\images\1-3-46-670589-33-1-63705540012666937300001-5673688970564737961_png_jpg.rf.15cca2fecc5f56865de3eb405476b90d.jpg: 288x320 1 kidney_stone, 19.4ms
Speed: 3.0ms preprocess, 19.4ms inference, 2.1ms postprocess per image at shape (1, 3, 288, 320)

image 1/1 C:\Users\Yaswanth Reddy\KidneyStoneDetec

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt")  # Or your trained model .pt path



100%|██████████| 21.5M/21.5M [01:56<00:00, 193kB/s] 


Ultralytics 8.3.146  Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2050, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, pr

RuntimeError: Dataset 'data.yaml' error  'data.yaml' does not exist

In [28]:
model.train(
    data='kidney.yaml',
    epochs=30,
    imgsz=320,
    batch=8,
    device=0 if torch.cuda.is_available() else 'cpu',
    project='kidney-stone-yolo',
    name='yolov8s-classifier',
    pretrained=True
)

Ultralytics 8.3.146  Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2050, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=kidney.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s-classifier, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretra

train: Scanning C:\Users\Yaswanth Reddy\KidneyStoneDetection\dataset\train\labels.cache... 1054 images, 1 backgrounds, 0 corrupt: 100%|██████████| 1054/1054 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.10.0 ms, read: 16.65.7 MB/s, size: 17.3 KB)


val: Scanning C:\Users\Yaswanth Reddy\KidneyStoneDetection\dataset\valid\labels.cache... 123 images, 0 backgrounds, 0 corrupt: 100%|██████████| 123/123 [00:00<?, ?it/s]


Plotting labels to kidney-stone-yolo\yolov8s-classifier\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 320 train, 320 val
Using 8 dataloader workers
Logging results to kidney-stone-yolo\yolov8s-classifier
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        123        325      0.733        0.6      0.632      0.256



30 epochs completed in 0.156 hours.
Optimizer stripped from kidney-stone-yolo\yolov8s-classifier\weights\last.pt, 22.5MB
Optimizer stripped from kidney-stone-yolo\yolov8s-classifier\weights\best.pt, 22.5MB

Validating kidney-stone-yolo\yolov8s-classifier\weights\best.pt...
Ultralytics 8.3.146  Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2050, 4096MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  4.55it/s]


                   all        123        325      0.761      0.578      0.623      0.259
Speed: 0.2ms preprocess, 5.1ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to kidney-stone-yolo\yolov8s-classifier


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000021E3A5EFC50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [29]:
model = YOLO(r'C:\Users\Yaswanth Reddy\KidneyStoneDetection\model\kidney-stone-yolo\yolov8s-classifier\weights\best.pt') 
model.train(
    data='kidney.yaml',
    imgsz=320,
    batch=8,
    epochs=100,
    patience=15,
    augment=True,
    device=0 if torch.cuda.is_available() else 'cpu',
    project='kidney-stone-yolo',
    name='yolov8s-classifier',
)

Ultralytics 8.3.146  Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2050, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=kidney.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=C:\Users\Yaswanth Reddy\KidneyStoneDetection\model\kidney-stone-yolo\yolov8s-classifier\weights\best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s-classifier2, nbs=64, nms=False, opset=None, optimize=False, o

train: Scanning C:\Users\Yaswanth Reddy\KidneyStoneDetection\dataset\train\labels.cache... 1054 images, 1 backgrounds, 0 corrupt: 100%|██████████| 1054/1054 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.10.0 ms, read: 16.66.6 MB/s, size: 17.3 KB)


val: Scanning C:\Users\Yaswanth Reddy\KidneyStoneDetection\dataset\valid\labels.cache... 123 images, 0 backgrounds, 0 corrupt: 100%|██████████| 123/123 [00:00<?, ?it/s]


Plotting labels to kidney-stone-yolo\yolov8s-classifier2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 320 train, 320 val
Using 8 dataloader workers
Logging results to kidney-stone-yolo\yolov8s-classifier2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        123        325      0.814      0.563       0.64      0.247
EarlyStopping: Training stopped early as no improvement observed in last 15 epochs. Best results observed at epoch 33, best model saved as best.pt.
To update EarlyStopping(patience=15) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



48 epochs completed in 0.248 hours.
Optimizer stripped from kidney-stone-yolo\yolov8s-classifier2\weights\last.pt, 22.5MB
Optimizer stripped from kidney-stone-yolo\yolov8s-classifier2\weights\best.pt, 22.5MB

Validating kidney-stone-yolo\yolov8s-classifier2\weights\best.pt...
Ultralytics 8.3.146  Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2050, 4096MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  2.91it/s]


                   all        123        325      0.709      0.572      0.634      0.262
Speed: 0.2ms preprocess, 15.3ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to kidney-stone-yolo\yolov8s-classifier2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000021DD6B4CB00>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [31]:
from ultralytics import YOLO
import pandas as pd
import os

# Load the best model
model = YOLO(r'C:\Users\Yaswanth Reddy\KidneyStoneDetection\model\kidney-stone-yolo\yolov8s-classifier2\weights\best.pt')

# Inference source (folder with test images)
source_folder = r"C:\Users\Yaswanth Reddy\KidneyStoneDetection\dataset\test\images"  # <-- change this to your test folder
results = model.predict(source=source_folder, save=False)

# Prepare output data
output = []

for result in results:
    image_path = result.path
    image_name = os.path.basename(image_path)

    # Check if probs exist (only for classification models)
    if result.probs:
        probs = result.probs.data.tolist()
        predicted_index = probs.index(max(probs))
        predicted_class = model.names[predicted_index]
        confidence = max(probs)
    else:
        predicted_class = "Unknown"
        confidence = 0.0

    output.append({
        "Image": image_name,
        "Predicted Class": predicted_class,
        "Confidence": round(confidence, 4)
    })

# Save to CSV
df = pd.DataFrame(output)
df.to_csv("yolo_classification_results.csv", index=False)

print("✅ Results saved to yolo_classification_results.csv")



image 1/123 C:\Users\Yaswanth Reddy\KidneyStoneDetection\dataset\test\images\1-3-46-670589-33-1-63703718086120120200001-5487554579919763006_png_jpg.rf.9fd67251e99a47dbe83a5db6efe6c016.jpg: 288x320 1 kidney_stone, 70.0ms
image 2/123 C:\Users\Yaswanth Reddy\KidneyStoneDetection\dataset\test\images\1-3-46-670589-33-1-63705534438365105500001-5275982036206127404_png_jpg.rf.365c4daf2b772012fe47e07b9daec86e.jpg: 288x320 2 kidney_stones, 37.9ms
image 3/123 C:\Users\Yaswanth Reddy\KidneyStoneDetection\dataset\test\images\1-3-46-670589-33-1-63705540012666937300001-5673688970564737961_png_jpg.rf.15cca2fecc5f56865de3eb405476b90d.jpg: 288x320 1 kidney_stone, 9.4ms
image 4/123 C:\Users\Yaswanth Reddy\KidneyStoneDetection\dataset\test\images\1-3-46-670589-33-1-63705542123217653900001-5305208767418446842_png_jpg.rf.d6f32a0ac819e4f2a870edfc1ce8079b.jpg: 288x320 2 kidney_stones, 26.0ms
image 5/123 C:\Users\Yaswanth Reddy\KidneyStoneDetection\dataset\test\images\1-3-46-670589-33-1-6370554212325365600000

In [32]:
data = pd.read_csv("yolo_classification_results.csv")

In [34]:
data.head()

,Image,Predicted Class,Confidence
0,1-3-46-670589-33-1-63703718086120120200001-548...,Unknown,0.0
1,1-3-46-670589-33-1-63705534438365105500001-527...,Unknown,0.0
2,1-3-46-670589-33-1-63705540012666937300001-567...,Unknown,0.0
3,1-3-46-670589-33-1-63705542123217653900001-530...,Unknown,0.0
4,1-3-46-670589-33-1-63705542123253656000001-487...,Unknown,0.0


CUDA available: True


^C
Note: you may need to restart the kernel to use updated packages.
